Overview

Author: stephankokkas

This notebook defines a pipeline that tasks an input directory of audio files and converts them to images using mel-spectrogram transofrmation and preprocessing techniques.

In [1]:
# disable warnings to tidy up output
import warnings
warnings.filterwarnings("ignore")

# some basic libraries 
from platform import python_version
#import pandas as pd
#import seaborn as sns
import numpy as np
import os
import random
from os import listdir
from os.path import isfile, join
import shutil
import torch
from IPython.display import Audio
import pandas as pd

# plot support
import matplotlib.pyplot as plt

# tensorflow support
import tensorflow as tf
#import tensorflow_transform as tft
import tensorflow_io as tfio
#from tensorflow.contrib.framework.python.ops import audio_ops

# scipy
import scipy
from pydub import AudioSegment, effects

# turn off tensorflow warnings
tf.get_logger().setLevel('ERROR')

# turn off absl warnings
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# print system information
print('Python Version     : ', python_version())
print('TensorFlow Version : ', tf.__version__)

Python Version     :  3.7.3
TensorFlow Version :  2.10.0


In [2]:
# below code adapted from:
# https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(123)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
random.seed(123)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(1234)

In [3]:
def enforce_memory_limit(mem_mb):
  # enforce memory limit on GPU

  gpus = tf.config.experimental.list_physical_devices('GPU')
  if gpus:
    try:
      tf.config.experimental.set_virtual_device_configuration(
          gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=mem_mb)])
      print(f"vram limit set to {mem_mb}MB")
    except RuntimeError as e:
      print(e)
      
# enforce max 5GB memory on GPU for this notebook
enforce_memory_limit(5120)

vram limit set to 5120MB


## Preprocessing Pipeline

This pipeline will go through a root directory and find all the audio files that exist and are of accepted format. Then, depending on the params set, it with normalise, trim and split the data. Please ensure you specify the self.DATASET_PATH with the directory of the data, and the self._SET_OUTPUT_DIR with the location you want the output files to end up in.

In [4]:
class raw_file_pre_processing():
    def __init__(self) -> None:
        self.CLIP_LENGTH   = 5000   # only look at 5000 milliseconds of clip at the start of loaded audio file
        self.BITRATE = "160k"        # all the samples are converted to bit rate of 32000 (Samples/Second)
        self.labels = []
        self.raw_dirs = {}
        self.dataset = pd.DataFrame(columns=['Label', 'FileName', 'FileType', 'Directory'])
        self.TARGET_FORMAT = 'flac'
        self.ACCEPTED_FORMAT = ['.mp3', '.flac', '.aiff', '.mp4', '.m4a', '.wav', '.ogg']
        self.CLEAN_DIR_ = True
        self.TRAIN_SPLIT      = 0.80
        self.TEST_SPLIT       = 0.01
        self.VALIDATION_SPLIT = 0.19
        self.split_file_names = {}

        # WIN
        #self.DATASET_PATH = 'C:\\Users\\steph\\Documents\\birdclef2022\\'
        # MAC
        self.DATASET_PATH  = 'D:\\Data\\b3\\'
        
        # make sure this is in the same format for either window or mac
        #WIN
        #self._SET_OUTPUT_DIR = 'C:\\Users\\steph\\Downloads\\'
        #MAC
        self._SET_OUTPUT_DIR = 'D:\\Data\\b3_out\\'
        
        
        if '/' in self.DATASET_PATH:
            self.DATASET_PATH = os.path.join('/', *self.DATASET_PATH.split('/'))
            self.OUTPUT_DIR = os.path.join('/', *self._SET_OUTPUT_DIR.split('/'), 'OUTPUT_raw_flac')
            if not os.path.exists(self.OUTPUT_DIR):
                os.mkdir(self.OUTPUT_DIR)
        elif '\\' in str(self.DATASET_PATH):
            self.DATASET_PATH = str(os.path.join(*self.DATASET_PATH.split('\\'))).replace(':', ':\\')
            self.OUTPUT_DIR = str(os.path.join(*self._SET_OUTPUT_DIR.split('\\'), 'OUTPUT_raw_flac')).replace(':', ':\\')
            if not os.path.exists(self.OUTPUT_DIR):
                os.makedirs(self.OUTPUT_DIR)   
                

        self.TRAIN_DIR = os.path.join(str(self.OUTPUT_DIR).replace('OUTPUT_raw_flac', ''), 'TRAIN_raw_flac')
        self.TEST_DIR = os.path.join(str(self.OUTPUT_DIR).replace('OUTPUT_raw_flac', ''), 'TEST_raw_flac')
        self.VALIDATION_DIR = os.path.join(str(self.OUTPUT_DIR).replace('OUTPUT_raw_flac', ''), 'VALIDATION_raw_flac')

        def clean_dir(self):
            if os.path.exists(self.OUTPUT_DIR):
                shutil.rmtree(self.OUTPUT_DIR)
            if os.path.exists(self.TRAIN_DIR):
                shutil.rmtree(self.TRAIN_DIR)
            if os.path.exists(self.TEST_DIR):
                shutil.rmtree(self.TEST_DIR)
            if os.path.exists(self.VALIDATION_DIR):
                shutil.rmtree(self.VALIDATION_DIR)
        if self.CLEAN_DIR_: 
            clean_dir(self)

    def handle_duplicate_files(self) -> bool:
        try:
            for root, dir, files in os.walk(self.DATASET_PATH):
                for _class_ in dir:
                    _file_ = [f.split('.')[0] for f in listdir(os.path.join(root, _class_)) if isfile(join(os.path.join(root, _class_), f))]
                    _set_ = set([x for x in _file_ if _file_.count(x) > 1])

                    if len(_set_) > 0:
                        print(f'Class {_class_} has the following duplicates: {_set_}. Will remove dupliactes...')
                        for _elem_ in _set_:
                            for dir_file in [f for f in listdir(os.path.join(root, _class_)) if isfile(join(os.path.join(root, _class_), f))]:
                                if _elem_ in dir_file:
                                    os.remove(os.path.join(root, _class_, dir_file))      
                break
            return True
        except Exception as e:
            print(e)
            print('Terminating...')
            return False
                    
    def get_raw_file_paths(self):
        if self.handle_duplicate_files():
            print(f'Looking for files... acceptable formats include: {self.ACCEPTED_FORMAT}')
            for root, dir, files in os.walk(self.DATASET_PATH):
                if dir == []:
                    tmp_lable = os.path.split(root)[-1]
                    tmp_file_dir = []
                    tmp_filename_ = []
                    for file in files:
                        for ext in self.ACCEPTED_FORMAT:
                            if ext in str(file):
                                tmp_file_dir.append(os.path.join(root, file))
                                tmp_filename_.append(str(str(os.path.split(os.path.join(root, file))[-1]).split('.')[0]).replace('_', ''))

                            
                    self.raw_dirs.update({tmp_lable:tmp_file_dir})
                    self.split_file_names.update({tmp_lable:tmp_filename_})

            for key in self.raw_dirs:
                print(f'FOUND: {key} -> {len(self.raw_dirs[key])}')

    def audio_preprocessing(self, TRIM_AUDIO:bool = False, 
                                  NORM_AUDIO:bool = False):
        print('\nConvering audio files....\n')
        if not os.path.exists(self.OUTPUT_DIR):
            os.makedirs(self.OUTPUT_DIR)

        
        for key, item in self.split_file_names.items():
            train = item[int(len(item) * .00) : int(len(item) * self.TRAIN_SPLIT)]
            vali = item[int(len(item) * self.TRAIN_SPLIT) : int(len(item) * (self.TRAIN_SPLIT + self.VALIDATION_SPLIT))]
            test = item[int(len(item) * (self.TRAIN_SPLIT + self.VALIDATION_SPLIT)) : int(len(item) * 1.00)]
            self.split_file_names.update({key: [train, vali, test]})

        for key, item in self.raw_dirs.items():
            print(f'Converting {key} data ->> ...')
            tmp_dir_key = os.path.join(self.OUTPUT_DIR, key)
            if not os.path.exists(tmp_dir_key):
                os.makedirs(tmp_dir_key)

            for dir in item:
                try:
                    # read file
                    tmp_file_name = str(os.path.split(dir)[-1].split('.')[0]).replace('_', '')
                    raw_sound = AudioSegment.from_file(dir, format=dir.split('.')[-1])

                    if NORM_AUDIO:
                        # normalise file
                        raw_sound = effects.normalize(raw_sound)

                    # trim file
                    if TRIM_AUDIO:
                        arr_split_file = [raw_sound[idx:idx + self.CLIP_LENGTH] for idx in range(0, len(raw_sound), self.CLIP_LENGTH)]             
                        for count_sample, sample in enumerate(arr_split_file):
                            # padding audio < 5s
                            if len(sample) < self.CLIP_LENGTH:
                                silence = AudioSegment.silent(duration=((self.CLIP_LENGTH-len(sample))))
                                sample = sample + silence  # Adding silence after the audio

                            # export raw file
                            tmp_raw_new_dir = os.path.join(tmp_dir_key, tmp_file_name + '_raw_trim_sample_' + str(count_sample) + '.' + self.TARGET_FORMAT)
                            sample.export(tmp_raw_new_dir, format=self.TARGET_FORMAT, bitrate=self.BITRATE, parameters = [])

                            new_row = pd.Series({"Label": key,
                                        "FileName": tmp_file_name + '_raw_trim_sample_' + str(count_sample) + '.' + self.TARGET_FORMAT,
                                        "FileType": self.TARGET_FORMAT,
                                        "Directory": tmp_raw_new_dir})
                            self.dataset = pd.concat([self.dataset, new_row.to_frame().T], ignore_index=True)
                    else:
                        tmp_raw_new_dir = os.path.join(tmp_dir_key, tmp_file_name + '_raw_' + '.' + self.TARGET_FORMAT)
                        raw_sound.export(tmp_raw_new_dir, format=self.TARGET_FORMAT, bitrate=self.BITRATE, parameters = [])

                        new_row = pd.Series({"Label": key,
                                    "FileName": tmp_file_name + '_raw_' + '.' + self.TARGET_FORMAT,
                                    "FileType": self.TARGET_FORMAT,
                                    "Directory": tmp_raw_new_dir})
                        self.dataset = pd.concat([self.dataset, new_row.to_frame().T], ignore_index=True)
                except Exception as e:
                    print(e)

    def train_test_split_fun(self):
        print(f'\nSplitting data into sub-directories Train, Test and Validation...')

        if not os.path.exists(self.TRAIN_DIR):
            os.mkdir(self.TRAIN_DIR)
        if not os.path.exists(self.TEST_DIR):
            os.mkdir(self.TEST_DIR)
        if not os.path.exists(self.VALIDATION_DIR):
            os.mkdir(self.VALIDATION_DIR)


        dict_keys = self.dataset['Label'].value_counts().to_dict()
        for key, item in dict_keys.items():
            if not os.path.exists(os.path.join(self.TRAIN_DIR, key)):
                os.mkdir(os.path.join(self.TRAIN_DIR, key))
            if not os.path.exists(os.path.join(self.TEST_DIR, key)):
                os.mkdir(os.path.join(self.TEST_DIR, key))
            if not os.path.exists(os.path.join(self.VALIDATION_DIR, key)):
                os.mkdir(os.path.join(self.VALIDATION_DIR, key))


        # self.dataset.to_csv(os.path.join(self.OUTPUT_DIR, 'raw_files.csv'))

        for index, row in self.dataset.iterrows():
            i = str(row.Directory)
            for key, item in self.split_file_names.items():
                if str(row.FileName).split('_')[0] in item[0]:
                    os.replace(i, i.replace('OUTPUT_raw_flac', 'TRAIN_raw_flac'))
                if str(row.FileName).split('_')[0] in item[1]:
                    os.replace(i, i.replace('OUTPUT_raw_flac', 'VALIDATION_raw_flac'))
                if str(row.FileName).split('_')[0] in item[2]:
                    os.replace(i, i.replace('OUTPUT_raw_flac', 'TEST_raw_flac'))

        shutil.rmtree(self.OUTPUT_DIR)

In [5]:
data_preprocessing_pipeline = raw_file_pre_processing()

data_preprocessing_pipeline.get_raw_file_paths()

print('\nThis next process will take approx 20 mins for the current bird dataset depending on the speed of your computer')
data_preprocessing_pipeline.audio_preprocessing(TRIM_AUDIO=True, NORM_AUDIO=True)
data_preprocessing_pipeline.train_test_split_fun()

Looking for files... acceptable formats include: ['.mp3', '.flac', '.aiff', '.mp4', '.m4a', '.wav', '.ogg']
FOUND: Acanthagenys rufogularis -> 7
FOUND: Acanthiza apicalis -> 5
FOUND: Acanthiza chrysorrhoa -> 13
FOUND: Acanthiza lineata -> 5
FOUND: Acanthiza nana -> 22
FOUND: Acanthiza pusilla -> 21
FOUND: Acanthiza reguloides -> 23
FOUND: Acanthiza uropygialis -> 2
FOUND: Acanthorhynchus tenuirostris -> 9
FOUND: Accipiter cirrocephalus -> 2
FOUND: Accipiter fasciatus -> 3
FOUND: Accipiter cooperi -> 1
FOUND: Acrocephalus australis -> 20
FOUND: Aegotheles cristatus -> 73
FOUND: Aidemosyne modesta -> 14
FOUND: Alauda arvensis European Skylark -> 54
FOUND: Alisterus scapularis -> 11
FOUND: Amytornis purnelli -> 1
FOUND: Anas superciliosa -> 5
FOUND: Anhinga novaehollandiae -> 2
FOUND: Anseranas semipalmata -> 3
FOUND: Anthochaera carunculata -> 47
FOUND: Anthochaera chrysoptera -> 17
FOUND: Anthochaera phrygia -> 139
FOUND: Anthus novaeseelandiae -> 7
FOUND: Antigone antigone -> 1
FOUND: 

KeyboardInterrupt: 

## Melspectrogram Pipeline

This pipeline will get all the new flac files, convert them to tfio tensors, convert to spectrograms, convert again to mel-spectrograms. It will then save the tensors as .pt files which can be read again

In [ ]:
class mel_spectrogram_pipeline():
    def __init__(self, output_directory) -> None:
        self.target_dir = ''
        self.labels = []
        self.augmented_dirs = {}
        self.OUTPUT_DIR = {}
        self.ACCEPTED_FORMAT = '.flac'

        # do not change, this will be brought across from the previous pipeline
        self._SET_OUTPUT_DIR = output_directory
        if '/' in self._SET_OUTPUT_DIR:
            self.DATASET_PATH = self._SET_OUTPUT_DIR
            self.TENSOR_OUTPUT_DIR = os.path.join(self.DATASET_PATH, 'OUTPUT_tensors')
        elif '\\' in self._SET_OUTPUT_DIR:
            self.DATASET_PATH = self._SET_OUTPUT_DIR
            self.TENSOR_OUTPUT_DIR = os.path.join(self.DATASET_PATH, 'OUTPUT_tensors')

        self.NFFT = 512
        self.WINDOW = 512
        self.STRIDE = 512
        self.SAMPLE_RATE = int(44100/2)
        self.MELS = 128
        self.FMIN = 0
        self.FMAX = int(self.SAMPLE_RATE)/2
        self.TOP_DB = 80

    
    def clean_dirs(self) -> None:
        print('Cleaning tensor directory')
        if os.path.exists(self.TENSOR_OUTPUT_DIR):
            shutil.rmtree(self.TENSOR_OUTPUT_DIR)

    def get_output_dir(self) -> None:
        print('Finding all pre-processed files')
        for root, dir, files in os.walk(self.DATASET_PATH):
            if "TRAIN_raw_flac" not in dir:
                if "OUTPUT" not in dir:
                    raise ValueError('Cant find any directories with pre-processed data. Looking for OUTPUT or TRAIN, TEST, and VAIDATION')
                else:
                    self.OUTPUT_DIR.update({"OUTPUT": os.path.join(root, "OUTPUT")})
            else:
                if "TEST_raw_flac" in dir and "VALIDATION_raw_flac" in dir:
                    self.OUTPUT_DIR.update({"TRAIN": os.path.join(root, "TRAIN_raw_flac")})
                    self.OUTPUT_DIR.update({"TEST": os.path.join(root, "TEST_raw_flac")})
                    self.OUTPUT_DIR.update({"VALIDATION": os.path.join(root, "VALIDATION_raw_flac")})
            if not self.OUTPUT_DIR:
                raise ValueError('Cant find any directories with pre-processed data. Looking for OUTPUT or TRAIN, TEST, and VAIDATION')

            print(f'\nFound the following directories {self.OUTPUT_DIR}\n')
            break
            
    def get_preprocessed_files(self) -> None:
        self.get_output_dir()

        for key, item in self.OUTPUT_DIR.items():
            for root, dir, files in os.walk(item):
                if dir == []:
                    tmp_lable = str(os.path.split(root)[-1]) + "~" + key
                    tmp_file_dir = []
                    for file in files:
                        if self.ACCEPTED_FORMAT in str(file):
                            tmp_file_dir.append(os.path.join(root, file))
                            
                    self.augmented_dirs.update({tmp_lable:tmp_file_dir})

        for key in self.augmented_dirs:
            print(f'FOUND: {key} -> {len(self.augmented_dirs[key])}')

    def generate_mel_spectrograms(self, MEL_SPECTRO:bool = False, 
                                        SHOW_PLOT:bool = False, 
                                        FREQ_MASK:bool = False, 
                                        TIME_MASK:bool = False,
                                        TORCH_EXPORT:bool = False, 
                                        TFIO_EXPORT:bool = False) -> None:
                                        
        print(f'\nGenerating tensors... \n')
        for key, item in self.augmented_dirs.items():
            for dir in item:
                tmp_label = key.split('~')[0]
                tmp_set = key.split('~')[1]
                if not os.path.exists(os.path.join(self.TENSOR_OUTPUT_DIR)):
                    os.mkdir(os.path.join(self.TENSOR_OUTPUT_DIR))
                if not os.path.exists(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set)):
                    os.mkdir(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set))
                if not os.path.exists(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label)):
                    os.mkdir(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label))
       
                file_contents=tf.io.read_file(dir)
                try:
                    tmp_audio_t = tfio.audio.decode_flac(input=file_contents, dtype=tf.int16)
                except:
                    tmp_audio_t = tfio.audio.decode_flac(input=file_contents, dtype=tf.int32)
                    
                tmp_audio_t = tf.cast(tmp_audio_t, tf.float32)
                    
                tmp_audio_t = tfio.audio.resample(tmp_audio_t, tfio.audio.AudioIOTensor(dir)._rate.numpy(), self.SAMPLE_RATE)

                # Convert to spectrogram
                spectrogram = tfio.audio.spectrogram(
                    tmp_audio_t[:, 0], nfft=self.NFFT, window=self.WINDOW, stride=self.STRIDE)

                if SHOW_PLOT:
                    plt.figure()
                    plt.imshow(tf.math.log(spectrogram).numpy())

                if MEL_SPECTRO:
                    # # Convert to mel-spectrogram
                    mel_spectrogram = tfio.audio.melscale(
                        spectrogram, rate=self.SAMPLE_RATE, mels=self.MELS, fmin=self.FMIN, fmax=self.FMAX)

                    if SHOW_PLOT:
                        plt.figure()
                        plt.imshow(tf.math.log(mel_spectrogram).numpy())

                    if TORCH_EXPORT:
                        torch.save(mel_spectrogram, str(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label, os.path.split(dir)[-1].split('.')[0])) + '_raw_mel_spectrogram.pt')
                    if TFIO_EXPORT:
                        tf.io.write_file(str(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label, os.path.split(dir)[-1].split('.')[0])) + '_raw_mel_spectrogram.pt', tf.io.serialize_tensor(mel_spectrogram))

                    if FREQ_MASK and "TEST" not in dir and "VALIDATION" not in dir:
                        freq_mask = tfio.audio.freq_mask(mel_spectrogram, param=10)
                        if TORCH_EXPORT:
                            torch.save(freq_mask, str(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label, os.path.split(dir)[-1].split('.')[0])) + '_freq_mask_mel_spectrogram.pt')
                        if TFIO_EXPORT:
                            tf.io.write_file(str(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label, os.path.split(dir)[-1].split('.')[0])) + '_freq_mask_mel_spectrogram.pt', tf.io.serialize_tensor(mel_spectrogram))
                    
                    if TIME_MASK and "TEST" not in dir and "VALIDATION" not in dir:
                        time_mask = tfio.audio.time_mask(mel_spectrogram, param=10)
                        if TORCH_EXPORT:
                            torch.save(time_mask, str(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label, os.path.split(dir)[-1].split('.')[0])) + '_time_mask_mel_spectrogram.pt')
                        if TFIO_EXPORT:
                            tf.io.write_file(str(os.path.join(self.TENSOR_OUTPUT_DIR, tmp_set, tmp_label, os.path.split(dir)[-1].split('.')[0])) + '_time_mask_mel_spectrogram.pt', tf.io.serialize_tensor(mel_spectrogram))

        print("\nTensors complete.\n")

In [ ]:
data_spectro_pipeline = mel_spectrogram_pipeline(data_preprocessing_pipeline._SET_OUTPUT_DIR)

data_spectro_pipeline.clean_dirs()
data_spectro_pipeline.get_preprocessed_files()

print('\nThis process will take approx 5-10 mins to complete depending on the power of your PC')
data_spectro_pipeline.generate_mel_spectrograms(MEL_SPECTRO=True, SHOW_PLOT=False, FREQ_MASK=True, TIME_MASK=True, TORCH_EXPORT=False, TFIO_EXPORT=True)

Cleaning tensor directory
Finding all pre-processed files

Found the following directories {'TRAIN': 'D:\\Data\\bc_out\\TRAIN_raw_flac', 'TEST': 'D:\\Data\\bc_out\\TEST_raw_flac', 'VALIDATION': 'D:\\Data\\bc_out\\VALIDATION_raw_flac'}

FOUND: brant~TRAIN -> 759
FOUND: jabwar~TRAIN -> 763
FOUND: sheowl~TRAIN -> 915
FOUND: spodov~TRAIN -> 610
FOUND: wiltur~TRAIN -> 1081
FOUND: brant~TEST -> 9
FOUND: jabwar~TEST -> 13
FOUND: sheowl~TEST -> 8
FOUND: spodov~TEST -> 23
FOUND: wiltur~TEST -> 1
FOUND: brant~VALIDATION -> 228
FOUND: jabwar~VALIDATION -> 105
FOUND: sheowl~VALIDATION -> 123
FOUND: spodov~VALIDATION -> 185
FOUND: wiltur~VALIDATION -> 176

This process will take approx 5-10 mins to complete depending on the power of your PC

Generating tensors... 


Tensors complete.



## Pipeline to load data into memory for model training

This pipeline will load all the tensors into a train, test, and validation data structure and prepare it for inputs into a model for training

In [ ]:
class train_test_vali_pipeline():
    def __init__(self, output_directory) -> None:
        self.DATASET_PATH  = output_directory
        self.TENSOR_OUTPUT_DIR  = None

        if '/' in self.DATASET_PATH:
            self.TENSOR_OUTPUT_DIR = os.path.join(self.DATASET_PATH, 'OUTPUT_tensors')
        elif '\\' in self.DATASET_PATH:
            self.TENSOR_OUTPUT_DIR = os.path.join(self.DATASET_PATH, 'OUTPUT_tensors')

        self.VALID_FILES = False

        self.PATHS = []
        self.train_data = self.test_data = self.vali_data = pd.DataFrame(columns=['Label', 'Tensor'])

    
    def check_valid_dirs(self) -> None:
        def contains_test(arr):
            if any("TEST" in item for item in arr): return True
            return False
        def contains_train(arr):
            if any("TRAIN" in item for item in arr): return True
            return False
        def contains_vali(arr):
            if any("VALIDATION" in item for item in arr): return True
            return False

        print('Checking to find train, test and vali directories inside tensors folder...')
        for root, dir, files in os.walk(self.TENSOR_OUTPUT_DIR):
            for i in dir:
                self.PATHS.append(os.path.join(root, i))
            if contains_test(dir) and contains_train(dir) and contains_vali(dir):
                self.VALID_FILES = True
                print('PASS')
            else:
                raise ValueError('Cannot find folders from previous pipline which include train, test and validation directories')
            break

    def load_data(self, LOAD_RAW:bool = False, 
                        LOAD_FREQ:bool = False, 
                        LOAD_TIME:bool = False, 
                        TO_CSV:bool = False, 
                        TORCH_LOAD:bool = False, 
                        TFIO_LOAD:bool = False) -> None:

        for path in self.PATHS:
            if 'TEST' in str(path) or 'TRAIN' in str(path) or 'VALIDATION' in str(path):
                for root, dir, file in os.walk(path):
                    for tmp_label in dir:  
                        for file in [f for f in listdir(os.path.join(root, tmp_label)) if isfile(join(os.path.join(root, tmp_label), f))]:
                            if '_raw_mel' in str(file) and LOAD_RAW:
                                if TORCH_LOAD:
                                    tmp_data = torch.load(os.path.join(path, tmp_label, file))
                                if TFIO_LOAD:
                                    tmp_data = tf.io.parse_tensor(tf.io.read_file(os.path.join(path, tmp_label, file)), tf.float32)
                                    
                                new_row = pd.Series({"Label": tmp_label,
                                                    "Tensor": tmp_data})
                                if "TRAIN" in path:
                                    self.train_data = pd.concat([self.train_data, new_row.to_frame().T], ignore_index=True)
                                elif "TEST" in path:
                                    self.test_data = pd.concat([self.test_data, new_row.to_frame().T], ignore_index=True)
                                elif "VALIDATION" in path:
                                    self.vali_data = pd.concat([self.vali_data, new_row.to_frame().T], ignore_index=True)
                            elif '_freq_mask' in str(file) and LOAD_FREQ:
                                if TORCH_LOAD:
                                    tmp_data = torch.load(os.path.join(path, tmp_label, file))
                                if TFIO_LOAD:
                                    tmp_data = tf.io.parse_tensor(tf.io.read_file(os.path.join(path, tmp_label, file)), tf.float32)

                                new_row = pd.Series({"Label": tmp_label,
                                                    "Tensor": tmp_data})
                                if "TRAIN" in path:
                                    self.train_data = pd.concat([self.train_data, new_row.to_frame().T], ignore_index=True)
                                elif "TEST" in path:
                                    self.test_data = pd.concat([self.test_data, new_row.to_frame().T], ignore_index=True)
                                elif "VALIDATION" in path:
                                    self.vali_data = pd.concat([self.vali_data, new_row.to_frame().T], ignore_index=True)
                            elif '_time_mask' in str(file) and LOAD_TIME:
                                if TORCH_LOAD:
                                    tmp_data = torch.load(os.path.join(path, tmp_label, file))
                                if TFIO_LOAD:
                                    tmp_data = tf.io.parse_tensor(tf.io.read_file(os.path.join(path, tmp_label, file)), tf.float32)
                                    
                                new_row = pd.Series({"Label": tmp_label,
                                                    "Tensor": tmp_data})
                                if "TRAIN" in path:
                                    self.train_data = pd.concat([self.train_data, new_row.to_frame().T], ignore_index=True)
                                elif "TEST" in path:
                                    self.test_data = pd.concat([self.test_data, new_row.to_frame().T], ignore_index=True)
                                elif "VALIDATION" in path:
                                    self.vali_data = pd.concat([self.vali_data, new_row.to_frame().T], ignore_index=True)

        if TO_CSV:
            self.train_data.to_csv(os.path.join(self.DATASET_PATH, 'train_df.csv'))
            self.test_data.to_csv(os.path.join(self.DATASET_PATH, 'test_df.csv'))
            self.vali_data.to_csv(os.path.join(self.DATASET_PATH, 'vali_df.csv'))

In [ ]:
model_data_pipeline = train_test_vali_pipeline(data_preprocessing_pipeline._SET_OUTPUT_DIR)

model_data_pipeline.check_valid_dirs()

print('This process will take approx 1 minute')
model_data_pipeline.load_data(LOAD_RAW=True, LOAD_FREQ=True, LOAD_TIME=True, TO_CSV=False, TORCH_LOAD=False, TFIO_LOAD=True)

print("Finished")
print(f'Train Shape: {model_data_pipeline.train_data.shape}, Test Shape: {model_data_pipeline.test_data.shape}, Validation Shape: {model_data_pipeline.vali_data.shape}') 

Checking to find train, test and vali directories inside tensors folder...
PASS
This process will take approx 1 minute
Finished
Train Shape: (12384, 2), Test Shape: (54, 2), Validation Shape: (817, 2)
